In [1]:
import sys,os,math
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib notebook
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, accuracy_score, auc, make_scorer
from scipy import stats
from numpy import interp
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold
from astropy.table import Table
import time
import astropy
import glob
import pickle

In [2]:
fnames = ['objid', 'wwE1',
 'wwE2',
 'wwFPSFKronDist',
 'wwFPSFApDist',
 'wwFPSFApRatio',
 'wwFPSFKronRatio',
 'wwFPSFflxR5Ratio',
 'wwFPSFflxR6Ratio',
 'wwFPSFflxR7Ratio']
features = ['wwE1',
 'wwE2',
 'wwFPSFKronDist',
 'wwFPSFApDist',
 'wwFPSFApRatio',
 'wwFPSFKronRatio',
 'wwFPSFflxR5Ratio',
 'wwFPSFflxR6Ratio',
 'wwFPSFflxR7Ratio']


In [3]:
data_dir = "/home/xhall/Documents/PS1_MLData/"
data_files = np.sort(glob.glob(data_dir + "/*.fit"))

In [4]:
classifier = pickle.load(open("model.model", 'rb'))

In [5]:
classifier

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=3,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=900,
                       n_jobs=-1, oob_score=False, random_state=20, verbose=0,
                       warm_start=False)

In [6]:
counter = 0
mldata_nona_list = []
for i in data_files:
    mldata = (Table.read(i)[fnames]).to_pandas()
    mldata_nona_list.append(mldata.dropna())

In [7]:
mldata_nona = pd.concat(mldata_nona_list, ignore_index = "True")

In [ ]:
mldata_nona.to_hdf(data_dir + "PS1_MLData.hdf", "data_table")

In [11]:
sums = 0
for i in mldata_nona_list:
    sums += len(i)
sums

144870754

In [6]:
mldata_nona = pd.read_hdf(data_dir + "PS1_MLData.hdf")

In [7]:
objids = np.asarray(mldata_nona["objid"])
X_hst = np.asarray(mldata_nona[features])

In [9]:
N_split = 10
split_data = np.array_split(X_hst, N_split)
split_predicted = []
for data in split_data:
    split_predicted.append(classifier.predict_proba(data))

predictions = np.concatenate(split_predicted)

In [16]:
predictions[:,1]

array([0.40230952, 0.38779101, 0.2814246 , ..., 0.39169974, 0.87128527,
       0.15198633])

In [14]:
len(df_out)

144870754

In [17]:
df_out = pd.DataFrame(data=objids, columns=["objid"])
df_out["rf_score"] = predictions[:,1]
df_out.to_hdf(data_dir + "class_table.hdf", "class_table")

In [4]:
class_table = pd.read_hdf(data_dir + "class_table.hdf")

In [9]:
class_table.to_csv(data_dir + "class_table.ascii", sep = " ", index=False)

In [14]:
file = open(data_dir + "class_table.ascii", "w")
file.write("objid score\n")

12

In [26]:
for i in Table.from_pandas(class_table):
    file.write("%s %5.3f\n" % (i["objid"], i["rf_score"]))
file.close()

In [24]:
"%s %5.3f\n" % (i["objid"], i["rf_score"])

'150333149759523988 0.402\n'

In [22]:
for i in Table.from_pandas(class_table):
    print(i)
    break

      objid              rf_score     
------------------ -------------------
150333149759523988 0.40230952380952373
